In [ ]:
!pip install faker mysql-connector-python
!pip install langchain_openai langchain_community langchain pymysql chromadb -q


In [ ]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Set environment variables (you may want to replace these with actual values)
os.environ["OPENAI_API_KEY"] = "sk-proj-nMxxxxxx5JFqWbdulwQhX5vHuy"

# Database connection setup. Can use sql.freedb.tech, and download sample data
# Sample data from https://www.mysqltutorial.org/getting-started-with-mysql/mysql-sample-database/
# sql.freedb.tech free version have its limit, max 1000 execution per user. Copy the sample data, sql
# excute directly in the sql.freedb.tech console. Below code, i change the Table name, adjust the SQL for the table name.


db_user = "freedb_ywk02"
db_password = "!tFpxxxxxx#9YJ"
db_host = "sql.freedb.tech:3306"
db_name = "freedb_xxxx"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

# Set up the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
generate_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

#printing sql
debug_on = False  #True / False to print out extra data for troubleshooting

#To enable Langsmith for tracking. Can change to "false" to disable it
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_sk_cd95xxxxxxxxx4a52fd72_9ee9e5998c" #Service API Key


# Function to extract SQL query from AI output
def extract_sql_query(output):
    sql_keywords = ["SELECT", "UPDATE", "INSERT", "DELETE", "CREATE", "DROP", "ALTER"]
    for keyword in sql_keywords:
        start_pos = output.upper().find(keyword)
        if start_pos != -1:
            query_end = output.find(";", start_pos)
            return output[start_pos:query_end+1].strip() if query_end != -1 else output[start_pos:].strip()
    return None

# Prompt template to rephrase the answer
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:"""
)

rephase_answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=generate_query)
    .assign(extracted_query=lambda result: extract_sql_query(result["query"]))
    .assign(result=itemgetter("extracted_query") | execute_query) | rephase_answer
)


def chat_loop():
    print("To exit, type /end\n")
    while True:
        question_for_ai = input("Ask a question: ")
        if question_for_ai.strip().lower() == '/end':
            print("Exiting conversation...")
            break

        # Generate the SQL query from the question
        query_output = generate_query.invoke({"question": question_for_ai})
        if debug_on: print("Before Extract:")
        if debug_on: print(query_output)
        if debug_on: print("\n")
        query = extract_sql_query(query_output)

        if query:
            if debug_on: print("Extracted SQL Query:")
            if debug_on: print(query)
            if debug_on: print("\n")

            try:
                result = execute_query.invoke(query)

                # Check if the result is empty or None
                if not result:
                    print("It seems there’s no information available for that query.\n")
                else:
                    # Get the rephrased answer for the user
                    final_answer = rephase_answer.invoke({
                        "question": question_for_ai,
                        "query": query,
                        "result": result
                    })
                    print("Answer:", final_answer)
                    print("\n")

            except Exception as e:
                # Handle the error and print a user-friendly message
                print("Unable to process the question. Please try asking something else.\n")
                if debug_on: print("Error details:", e)  # Optional for debugging

        else:
            print("Hmm, I couldn't find the information you're asking for. Could you try phrasing it differently?\n")

# Start the chat loop
chat_loop()
